In [38]:
import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

In [32]:
df = pd.read_csv("deliveries-2.csv")
df.shape

(29083, 21)

In [15]:
df.head()

,user_id,can_select_unattended_delivery,floor,has_elevator,can_use_trolley,is_unattended_delivery,delivery_date,slot_start_time,slot_end_time,num_boxes_dry,...,num_bags_frozen,num_products,products_weight,street,street_number,zip_code,zip_place,area_segment,municipality,service_time
0,bf118b2b678a7fc7c87fbf976ffa8ed3033dea98,True,NaN,NaN,NaN,True,2021-03-01,2021-03-01 15:00:00+00:00,2021-03-01 20:00:00+00:00,1.0,...,1.0,38.0,16.12,820963425f8533458b049c6a321d9cdec324800b,32,259e58e1899790724f5bde68f6c687293fce64d1,e6d3f74639a9c1072763a89aa1f65defeb520c17,8766486fdc50fdbfffd5df9bea38fb731627271b,72fad23df48f54abdf674cd2c3b08b936ae42c35,7.0
1,ff1a01d243afb343a459080ffdddeba630c46cc5,False,1.0,False,True,False,2020-11-12,2020-11-12 08:00:00+00:00,2020-11-12 13:00:00+00:00,2.0,...,0.0,22.0,18.76,377d3a3c0981102ac51ca497f6c54bc4fc54f777,4,259e58e1899790724f5bde68f6c687293fce64d1,e6d3f74639a9c1072763a89aa1f65defeb520c17,10e38dfb9188746ed84129990029be2e70aafef6,72fad23df48f54abdf674cd2c3b08b936ae42c35,9.0
2,ff1a01d243afb343a459080ffdddeba630c46cc5,False,1.0,False,True,False,2020-12-09,2020-12-09 15:00:00+00:00,2020-12-09 20:00:00+00:00,2.0,...,1.0,22.0,10.82,377d3a3c0981102ac51ca497f6c54bc4fc54f777,4,259e58e1899790724f5bde68f6c687293fce64d1,e6d3f74639a9c1072763a89aa1f65defeb520c17,10e38dfb9188746ed84129990029be2e70aafef6,72fad23df48f54abdf674cd2c3b08b936ae42c35,10.0
3,ff1a01d243afb343a459080ffdddeba630c46cc5,False,1.0,False,True,False,2020-10-22,2020-10-22 14:00:00+00:00,2020-10-22 19:00:00+00:00,2.0,...,1.0,26.0,18.97,377d3a3c0981102ac51ca497f6c54bc4fc54f777,4,259e58e1899790724f5bde68f6c687293fce64d1,e6d3f74639a9c1072763a89aa1f65defeb520c17,10e38dfb9188746ed84129990029be2e70aafef6,72fad23df48f54abdf674cd2c3b08b936ae42c35,9.0
4,e297e5a8a8f8cceb464325dea7e13391b6f5f280,True,NaN,NaN,NaN,True,2021-01-19,2021-01-19 08:00:00+00:00,2021-01-19 13:00:00+00:00,3.0,...,1.0,63.0,58.49,f87306cc3aef9415b0e836f9cf4b05f2c0a66ff7,20,259e58e1899790724f5bde68f6c687293fce64d1,e6d3f74639a9c1072763a89aa1f65defeb520c17,8d828ec7ea9a3c6faf91f036e3f222c18e0dc987,72fad23df48f54abdf674cd2c3b08b936ae42c35,22.0


In [16]:
# User ID explaines nothing
df.drop("user_id", axis = 1, inplace = True)
# Do not need zip_code and zip_place, so we remove zip_place
df.drop("zip_place", axis = 1, inplace = True)
df.head()

,can_select_unattended_delivery,floor,has_elevator,can_use_trolley,is_unattended_delivery,delivery_date,slot_start_time,slot_end_time,num_boxes_dry,num_boxes_cold,num_bags_frozen,num_products,products_weight,street,street_number,zip_code,area_segment,municipality,service_time
0,True,NaN,NaN,NaN,True,2021-03-01,2021-03-01 15:00:00+00:00,2021-03-01 20:00:00+00:00,1.0,1.0,1.0,38.0,16.12,820963425f8533458b049c6a321d9cdec324800b,32,259e58e1899790724f5bde68f6c687293fce64d1,8766486fdc50fdbfffd5df9bea38fb731627271b,72fad23df48f54abdf674cd2c3b08b936ae42c35,7.0
1,False,1.0,False,True,False,2020-11-12,2020-11-12 08:00:00+00:00,2020-11-12 13:00:00+00:00,2.0,1.0,0.0,22.0,18.76,377d3a3c0981102ac51ca497f6c54bc4fc54f777,4,259e58e1899790724f5bde68f6c687293fce64d1,10e38dfb9188746ed84129990029be2e70aafef6,72fad23df48f54abdf674cd2c3b08b936ae42c35,9.0
2,False,1.0,False,True,False,2020-12-09,2020-12-09 15:00:00+00:00,2020-12-09 20:00:00+00:00,2.0,1.0,1.0,22.0,10.82,377d3a3c0981102ac51ca497f6c54bc4fc54f777,4,259e58e1899790724f5bde68f6c687293fce64d1,10e38dfb9188746ed84129990029be2e70aafef6,72fad23df48f54abdf674cd2c3b08b936ae42c35,10.0
3,False,1.0,False,True,False,2020-10-22,2020-10-22 14:00:00+00:00,2020-10-22 19:00:00+00:00,2.0,1.0,1.0,26.0,18.97,377d3a3c0981102ac51ca497f6c54bc4fc54f777,4,259e58e1899790724f5bde68f6c687293fce64d1,10e38dfb9188746ed84129990029be2e70aafef6,72fad23df48f54abdf674cd2c3b08b936ae42c35,9.0
4,True,NaN,NaN,NaN,True,2021-01-19,2021-01-19 08:00:00+00:00,2021-01-19 13:00:00+00:00,3.0,2.0,1.0,63.0,58.49,f87306cc3aef9415b0e836f9cf4b05f2c0a66ff7,20,259e58e1899790724f5bde68f6c687293fce64d1,8d828ec7ea9a3c6faf91f036e3f222c18e0dc987,72fad23df48f54abdf674cd2c3b08b936ae42c35,22.0


In [37]:
# Now we want to deal with missing values
# proportion_of_missing_data
df[df.isna().any(axis=1)].shape[0]/df.shape[0]
# We use simple imputer

0.1735721899391397

In [8]:
X, y = df.drop("service_time", axis=1), df["service_time"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)